In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import pandas as pd
import numpy as np
import io
import random
import math
import scipy.stats
import matplotlib.pylab as plt

C:\Users\devin\AppData\Local\Temp\ipykernel_37572\621582838.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
# Initialize year variable
year = 2022 #(2019, 2021, 2022, 2023)

In [3]:
# Read in data based on the year
if year == 2019:
    data = pd.read_csv('data/18_19tourneycs.csv')
elif year == 2021:
    data = pd.read_csv('data/20_21tourneycs.csv')
elif year == 2022:
    data = pd.read_csv('data/21_22tourneycs.csv')
data.head()

,Region,Seed,School,SRS
0,1,1,Baylor,21.73
1,1,16,Norfolk State,-3.91
2,1,8,North Carolina,15.50
3,1,9,Marquette,11.36
4,1,4,UCLA,19.30


In [4]:
# Rename columns of data DataFrame
data.columns = ['Region', 'Seed', 'Team', 'SRS']
data

,Region,Seed,Team,SRS
0,1,1,Baylor,21.73
1,1,16,Norfolk State,-3.91
2,1,8,North Carolina,15.50
3,1,9,Marquette,11.36
4,1,4,UCLA,19.30
...,...,...,...,...
59,4,11,Iowa State,12.38
60,4,7,USC,12.50
61,4,10,Miami,11.22
62,4,2,Auburn,19.20


In [5]:
num_of_sims = 1000  # Number of simulations
teams = data.to_numpy().tolist()  # Convert DataFrame to a nested list
team_dict = {}  # Dictionary to store team data

# Iterate through each team
for i in range(0, len(teams)):
    # Initialize an empty list for each team in the team_dict
    team_dict[teams[i][2]] = [0, 0, 0, 0, 0, 0, 0]
    # The team name is accessed using teams[i][2], assuming the team name is in the third column of the data

In [6]:
"""
Given a list of teams in a specific round and the round number,
find_winners randomly determines the winners of each matchup based on the
SRS difference between the teams and returns a list of the winners
"""
def find_winners(input_list, round):

    winners = []

    # Iterate over the input list of teams
    for i in range(0, len(input_list)):
        # Skips if the index is odd (represents the second team in a matchup)
        if i % 2 != 0:
            pass
        else:
            # Get the first team and the second team in the matchup
            team1 = input_list[i]
            team2 = input_list[i+1]

            # Calculate the SRS difference between the teams
            srs_diff = team1[3] - team2[3]

            # Calculate the win probability of Team 1 using the SRS difference
            team1_prob = scipy.stats.norm(0, 10.36).cdf(srs_diff)

            # Generate a random draw between 0 and 1
            draw = random.random()

            # Determine the winner based on the draw and win probability
            if draw <= team1_prob:
                # Team 1 wins
                winners.append(team1)
                team_dict[team1[2]][round] += 1
            else:
                # Team 2 wins
                winners.append(team2)
                team_dict[team2[2]][round] += 1

    return winners


In [7]:
"""
Prints out the bracket representation of the tournament rounds
based on the provided round_list
"""
def bracket_printout(round_list):

    for i in range(0, len(round_list)):
        temp_list = round_list[i]

        # Iterate over the matchups in the current round
        for j in range(0, len(temp_list)):
            # Print the team seed, name, and SRS for each matchup
            print(str(temp_list[j][1]) + " " + temp_list[j][2] + " " + str(temp_list[j][3]))

            # Print an empty line after every even-indexed matchup
            if j % 2 != 0:
                print("")

        # Print empty lines between each round
        print("")
        print("")
        print("")


In [8]:
"""
Simulates a tournament by finding winners in each round and printing the bracket
"""
def tourney_sim(teams):
    # Increment the count of tournaments for each team
    for x in team_dict:
        team_dict[x][0] += 1

    # Find winners in the Round of 64
    round_32 = find_winners(teams, 1)

    # Find winners in the Round of 32
    sweet_16 = find_winners(round_32, 2)

    # Find winners in the Sweet 16
    elite_8 = find_winners(sweet_16, 3)

    # Find winners in the Elite 8
    final_4 = find_winners(elite_8, 4)

    # Find winners in the Final 4
    championship = find_winners(final_4, 5)

    # Find the winner in the Championship
    winner = find_winners(championship, 6)

    # Print the bracket for all rounds
    # Uncomment below to see bracket format of simulation. Heads up: it is a lot of output!
    # bracket_printout([teams, round_32, sweet_16, elite_8, final_4, championship, winner])


In [9]:
%%time
# Run the monte-carlo tournament simulation
for i in range(1, num_of_sims+1):
    tourney_sim(teams)

# Calculate the average statistics for each team
for x in team_dict:
    # Divide the count of wins in each round by the total number of simulations
    for i in range(0, len(team_dict[x])):
        team_dict[x][i] = team_dict[x][i] / num_of_sims

    # Print the team name and its average statistics
    print(x + " " + str(team_dict[x]))


Baylor [1.0, 0.997, 0.75, 0.484, 0.296, 0.129, 0.068]
Norfolk State [1.0, 0.003, 0.0, 0.0, 0.0, 0.0, 0.0]
North Carolina [1.0, 0.66, 0.188, 0.075, 0.028, 0.01, 0.004]
Marquette [1.0, 0.34, 0.062, 0.018, 0.004, 0.0, 0.0]
UCLA [1.0, 0.956, 0.665, 0.336, 0.168, 0.081, 0.036]
Akron [1.0, 0.044, 0.008, 0.0, 0.0, 0.0, 0.0]
St. Mary’s [1.0, 0.567, 0.189, 0.051, 0.016, 0.004, 0.002]
Indiana [1.0, 0.433, 0.138, 0.036, 0.013, 0.003, 0.0]
Purdue [1.0, 0.97, 0.616, 0.333, 0.173, 0.06, 0.024]
Yale [1.0, 0.03, 0.001, 0.0, 0.0, 0.0, 0.0]
Texas [1.0, 0.589, 0.241, 0.098, 0.036, 0.013, 0.006]
Virginia Tech [1.0, 0.411, 0.142, 0.056, 0.023, 0.005, 0.0]
Murray State [1.0, 0.337, 0.057, 0.011, 0.004, 0.001, 0.0]
San Francisco [1.0, 0.663, 0.204, 0.069, 0.024, 0.008, 0.004]
Kentucky [1.0, 0.954, 0.729, 0.433, 0.215, 0.105, 0.047]
St. Peter's [1.0, 0.046, 0.01, 0.0, 0.0, 0.0, 0.0]
Gonzaga [1.0, 0.994, 0.844, 0.685, 0.5, 0.361, 0.239]
Georgia State [1.0, 0.006, 0.0, 0.0, 0.0, 0.0, 0.0]
Boise State [1.0, 0.42

In [10]:
point_system = [0, 1, 2, 3, 5, 8, 13]  # Predefined point system for each round

expected_values = {}  # Dictionary to store the expected value for each team

# Calculate the expected value for each team
for team, probabilities in team_dict.items():
    expected_value = sum(probabilities[i] * point_system[i] for i in range(len(probabilities)))
    expected_values[team] = expected_value

# Sort teams by expected value in descending order
sorted_teams = sorted(expected_values.items(), key=lambda x: x[1], reverse=True)

# Print the expected value for each team
for team, expected_value in sorted_teams:
    print(f"{team}: {expected_value:.3f}")

Gonzaga: 13.232
Kansas: 9.921
Arizona: 8.690
Baylor: 7.345
Houston: 7.162
Kentucky: 6.237
Tennessee: 5.961
Auburn: 5.933
Duke: 5.716
Texas Tech: 5.679
UCLA: 5.250
Villanova: 4.980
Purdue: 4.858
Iowa: 4.310
LSU: 3.348
Uconn: 2.725
Illinois: 2.229
Arkansas: 2.221
Wisconsin: 2.108
Texas: 1.727
Providence: 1.545
North Carolina: 1.533
San Francisco: 1.514
Alabama: 1.462
Michigan: 1.326
St. Mary’s: 1.236
Michigan State: 1.222
USC: 1.220
Ohio State: 1.103
Loyola Chicago: 1.082
Memphis: 1.080
Virginia Tech: 1.018
San Diego State: 0.995
Iowa State: 0.962
Indiana: 0.906
Miami: 0.901
TCU: 0.890
Seton Hall: 0.745
Creighton: 0.722
Notre Dame: 0.693
Colorado State: 0.689
Davidson: 0.661
Boise State: 0.607
Marquette: 0.538
Murray State: 0.512
South Dakota State: 0.413
Vermont: 0.346
New Mexico State: 0.277
UAB: 0.260
Richmond: 0.223
Chattanooga: 0.187
Colgate: 0.157
St. Peter's: 0.066
Akron: 0.060
Delaware: 0.051
Montana State: 0.036
Yale: 0.032
Cal State Fullerton: 0.027
Jacksonville State: 0.027
Lo

In [11]:
# Convert dictionary to DataFrame
df_expected_values = pd.DataFrame.from_dict(expected_values, orient='index', columns=['Expected Value'])

# Sort teams by expected value in descending order
df_expected_values = df_expected_values.sort_values(by='Expected Value', ascending=False)

# Reset the index
df_expected_values = df_expected_values.reset_index().rename(columns={'index': 'Team'})

# Display the DataFrame
display(df_expected_values)

,Team,Expected Value
0,Gonzaga,13.232
1,Kansas,9.921
2,Arizona,8.690
3,Baylor,7.345
4,Houston,7.162
...,...,...
59,Longwood,0.017
60,Wright State,0.013
61,Georgia State,0.006
62,Texas Southern,0.005


In [12]:
# Merge the dataframes based on the 'Team' column
df_merged = data.merge(df_expected_values, on='Team')

# Display the merged DataFrame
df_merged.head()

,Region,Seed,Team,SRS,Expected Value
0,1,1,Baylor,21.73,7.345
1,1,16,Norfolk State,-3.91,0.003
2,1,8,North Carolina,15.50,1.533
3,1,9,Marquette,11.36,0.538
4,1,4,UCLA,19.30,5.250


In [13]:
# Define the mapping of seed values to cost values
seed_cost_mapping = {
    1: 75,
    2: 40,
    3: 25,
    4: 20,
    5: 17,
    6: 15,
    7: 12,
    8: 10,
    9: 9,
    10: 8,
    11: 7,
    12: 6,
    13: 5,
    14: 4,
    15: 3,
    16: 1,
}

# Add a new column 'Cost' to df_merged based on the 'Seed' column
df_merged['Cost'] = df_merged['Seed'].map(seed_cost_mapping)

# Display the merged DataFrame
df_merged.head()

,Region,Seed,Team,SRS,Expected Value,Cost
0,1,1,Baylor,21.73,7.345,75
1,1,16,Norfolk State,-3.91,0.003,1
2,1,8,North Carolina,15.50,1.533,10
3,1,9,Marquette,11.36,0.538,9
4,1,4,UCLA,19.30,5.250,20


In [14]:
# Define the knapsack function
def knapsack(items, budget):
    n = len(items)
    dp = [[0] * (budget + 1) for _ in range(n + 1)]

    for i in range(1, n + 1):
        for j in range(1, budget + 1):
            cost = items[i - 1]['Cost']
            value = items[i - 1]['Expected Value']
            if cost > j:
                dp[i][j] = dp[i - 1][j]
            else:
                dp[i][j] = max(dp[i - 1][j], value + dp[i - 1][j - cost])

    selected = []
    j = budget
    for i in range(n, 0, -1):
        if dp[i][j] != dp[i - 1][j]:
            selected.append(items[i - 1])
            j -= items[i - 1]['Cost']

    return selected

# Run the knapsack optimization on df_merged
selected_teams = knapsack(df_merged.to_dict('records'), budget=224)

# Create a new DataFrame for the selected teams
df_selected = pd.DataFrame(selected_teams)

# Display the selected teams and their information
display(df_selected)

# Calculate the total sum of the cost and expected value columns
total_cost = df_selected['Cost'].sum()
total_expected_value = df_selected['Expected Value'].sum()

# Print the total sums
print(f"Total Cost: {total_cost}")
print(f"Total Expected Value: {total_expected_value:.3f}")

,Region,Seed,Team,SRS,Expected Value,Cost
0,4,6,LSU,17.45,3.348,15
1,4,5,Iowa,18.90,4.310,17
2,4,13,South Dakota State,6.60,0.413,5
3,3,3,Tennessee,20.46,5.961,25
4,3,5,Houston,22.55,7.162,17
5,2,3,Texas Tech,20.08,5.679,25
6,2,1,Gonzaga,25.46,13.232,75
7,1,3,Purdue,19.15,4.858,25
8,1,4,UCLA,19.30,5.250,20


Total Cost: 224
Total Expected Value: 50.213
